介绍一些pandas的高级常用操作

数据读取  
header指出哪一行是列名  
na_values哪一个标识表示NaN值  

In [2]:
import pandas as pd
data = pd.read_csv('data.csv',header=0, na_values=["NA"])
print(data)
print(data.dtypes)


   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000
NumRooms    float64
Alley        object
Price         int64
dtype: object


DF.dtypes：获取DataFrame各列的数据类型，返回Series，包含列名及其对应的数据类型  
Series.index：返回的是Pandas的 Index 对象，它代表了 Series 中数据的标签（索引）集合

In [15]:
colum_dtypes=data.dtypes
colum=colum_dtypes.index
colum_dtypes,colum

(NumRooms    float64
 Alley        object
 Price         int64
 dtype: object,
 Index(['NumRooms', 'Alley', 'Price'], dtype='object'))

筛选出dtypes为数值类型的列(Series)，返回一个拷贝的DataFrame  

In [3]:
num_data=data.select_dtypes(include=["number"])
num_data

,NumRooms,Price
0,NaN,127500
1,2.0,106000
2,4.0,178100
3,NaN,140000


提取数值列名（不会拷贝数据！）

In [4]:
numeric_cols = data.select_dtypes(include=["number"]).columns
numeric_cols

Index(['NumRooms', 'Price'], dtype='object')

<pre>布尔索引  
    原理：用布尔数组（长度与数据维度一致）选择 True 对应位置的行或列，过滤掉 False 的位置。
    特点：
        布尔数组可以是：
            列表（list）
            NumPy 数组（numpy.ndarray）
            Pandas（pandas.Series）
        支持逻辑运算（&, |, ~）和比较运算（==, !=, >, < 等）。
在series和dataframe里都可以使用

在 Series 中使用布尔索引

In [7]:
s = pd.Series([10, 20, 30, 40], index=['a', 'b', 'c', 'd'])
mask = [True, False, True, False]  # 布尔数组
result = s[mask]  # 筛选 True 对应的位置
result

a    10
c    30
dtype: int64

使用条件表达式生成布尔 Series

In [8]:
mask = s > 20  # 生成布尔 Series
result = s[mask]  # 筛选大于 20 的值
result

c    30
d    40
dtype: int64

在 DataFrame 中使用布尔索引  
DataFrame 中，布尔索引本身仅支持一维布尔数组（或 Series）用于行筛选，不支持直接使用二维布尔数组进行同时行和列的筛选  
单列条件筛选

In [12]:
df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': ['x', 'y', 'z', 'w'],
    'C': [10.5, 20.3, 30.1, 40.7]
})
mask = df['A'] > 2  # 筛选 A 列大于 2 的行
result = df[mask]
df,mask,result

(   A  B     C
 0  1  x  10.5
 1  2  y  20.3
 2  3  z  30.1
 3  4  w  40.7,
 0    False
 1    False
 2     True
 3     True
 Name: A, dtype: bool,
    A  B     C
 2  3  z  30.1
 3  4  w  40.7)

多列条件组合  

In [13]:
mask = (df['A'] > 1) & (df['C'] < 40.0)  # A > 1 且 C < 40
result = df[mask]
mask,result

(0    False
 1     True
 2     True
 3    False
 dtype: bool,
    A  B     C
 1  2  y  20.3
 2  3  z  30.1)

使用 isin() 筛选特定值

In [14]:
mask = df['B'].isin(['x', 'z'])  # B 列是 'x' 或 'z' 的行
result = df[mask]
mask,result

(0     True
 1    False
 2     True
 3    False
 Name: B, dtype: bool,
    A  B     C
 0  1  x  10.5
 2  3  z  30.1)

组合使用dtypes,index,布尔索引

In [17]:
numeric_col_index=df.dtypes[df.dtypes!="object"].index
numeric_df=df[numeric_col_index]
df.dtypes!="object",numeric_col_index,numeric_df

(A     True
 B    False
 C     True
 dtype: bool,
 Index(['A', 'C'], dtype='object'),
    A     C
 0  1  10.5
 1  2  20.3
 2  3  30.1
 3  4  40.7)

填充NaN值

In [4]:
means=data[numeric_cols].mean()
data[numeric_cols]=data[numeric_cols].fillna(means)
data

,NumRooms,Alley,Price
0,3.0,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,3.0,NaN,140000


独热编码（One-Hot Encoding），并额外处理缺失值（NaN）  
get_dummies输入数据是array或者DataFrame  
dummy_na处理nan值

输入是array时，返回一个pandas的DataFrame,形状为(array.size,array中不同的值的个数)
把array中所有不同的内容提取出来作为列的标签  
标签内容在array中的索引值（可能不止出现过一次），映射到dataframe的该标签列所对应的index就是true,其余值为false

In [5]:
import numpy as np
color_data=np.array(["Red", "Green", None, "Green"])
dummies = pd.get_dummies(color_data,dummy_na=True)
print(color_data)
print(dummies)

['Red' 'Green' None 'Green']
   Green    Red    NaN
0  False   True  False
1   True  False  False
2  False  False   True
3   True  False  False


In [6]:
data = pd.DataFrame({
    "Color": ["Red", "Blue", None, "Blue"],
    "Size": [np.nan, "M", "L", np.nan],
    "Price": [10, 20, 30, 40]  # 数值列不会被编码
})
print(data)
encoded_data = pd.get_dummies(data, dummy_na=True)
print(encoded_data)

  Color Size  Price
0   Red  NaN     10
1  Blue    M     20
2  None    L     30
3  Blue  NaN     40
   Price  Color_Blue  Color_Red  Color_nan  Size_L  Size_M  Size_nan
0     10       False       True      False   False   False      True
1     20        True      False      False   False    True     False
2     30       False      False       True    True   False     False
3     40        True      False      False   False   False      True


pandas与numpy的互转  
Series、DataFrame 转NumPy数组  to_numpy  

In [7]:
df = pd.DataFrame({"A": [1, 2, 3], "B": ["x", "y", "z"]})
# 方法1: .values (旧方法，仍可用)
numpy_array1 = df.values
# 方法2: .to_numpy() (推荐，更明确)
numpy_array2 = df.to_numpy()
print(numpy_array2)

[[1 'x']
 [2 'y']
 [3 'z']]


In [8]:
s = pd.Series([1, 2, 3])
numpy_array = s.to_numpy()  # 或 s.values
print(numpy_array)  # 输出: [1 2 3]

[1 2 3]


 NumPy 数组转 DataFrame  
 直接用numpy新建df  
 直接用numpy新建Series

In [9]:
arr_df = np.array([[1, "x"], [2, "y"], [3, "z"]])
# 转换为 DataFrame
df = pd.DataFrame(arr_df, columns=["A", "B"])
print(df)

   A  B
0  1  x
1  2  y
2  3  z


In [10]:
arr_s = np.array([1, 2, 3])
s = pd.Series(arr_s)
print(s)

0    1
1    2
2    3
dtype: int64


构造tensor  
只能用numpy构造，所以一般先用pd读取数据处理完后to_numpy,再转tensor  
pd转numpy也可以pd.values直接转
注意tensor只能是数值类型，字符串需要进行处理  
as_tensor可传入tensor、numpy、list、tuple

In [23]:
import torch
input=pd.get_dummies(df)
print(input)
ts_a=torch.from_numpy(input.to_numpy())
ts_b=torch.as_tensor(input.values.tolist(),device="cuda:0")
ts_c=torch.from_numpy(input.values)
print(input.values)
print(ts_a)
print(ts_b)
print(ts_c)

   A  B
0  1  3
1  2  4
[[1 3]
 [2 4]]
tensor([[1, 3],
        [2, 4]])
tensor([[1, 3],
        [2, 4]], device='cuda:0')
tensor([[1, 3],
        [2, 4]])


df.apply方法  
<pre>用于对 DataFrame 的行或列应用自定义函数的强大方法，能够高效处理复杂数据操作，避免显式循环。  
核心功能
    沿轴应用函数：对 DataFrame 的行（axis=0）或列（axis=1）应用函数，返回处理后的结果。
    支持复杂操作：可处理多列交互、条件判断、数据转换等任务。
    返回类型灵活：根据函数逻辑和参数设置，返回 Series、DataFrame 或标量值。


参数                说明                                                        示例
func            要应用的函数（自定义或内置）                                    lambda x: x*2
axis            应用方向：0（行，默认）或 1（列）                               axis=1 遍历所有行，对每行的列操作
result_type     控制返回类型：'reduce'（Series）、'expand'（DataFrame）、）       result_type='expand'
                    'broadcast'（广播结果）或 None（自动推断
args            传递给 func 的额外位置参数	                                    args=(2,)
**kwargs        传递给 func 的额外关键字参数                                  kwargs={'scale': 1.5}

In [ ]:
df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
#参数row代表被遍历的axis(没有被选中的axis)对应的每个数据，这里是行
row_sum = df.apply(lambda row: row['A'] + row['B'], axis=1)
df,row_sum

(   A  B
 0  1  3
 1  2  4,
 0    4
 1    6
 dtype: int64)

In [20]:
col_mean = df.apply(lambda col: col.mean(), axis=0)
col_mean

A    1.5
B    3.5
dtype: float64

注意，以上求和与求均值是被遍历的轴对应的向量变成一个标量，df返回一个一维的 Series,属于降维  
lambda表达式如果返回一个被遍历的轴的数据对应长度的向量，则不会降维  

In [ ]:
nom_df=df.apply(lambda col:(col-col.mean())/col.std())
nom_df


,A,B
0,-0.707107,-0.707107
1,0.707107,0.707107


使用 apply() + result_type='expand'（返回 DataFrame）  
result_type='expand' 强制返回 DataFrame

In [22]:
def process_col(col):
    return pd.Series([col.mean(), col.sum()], index=['mean', 'sum'])

result = df.apply(process_col, axis=0, result_type='expand')
df,result

(   A  B
 0  1  3
 1  2  4,
         A    B
 mean  1.5  3.5
 sum   3.0  7.0)